### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
# загрузка необходимой библиотеки
import pandas as pd
# загрузка базы данных
data = pd.read_csv('credit_approval.csv')

In [2]:
# вывод первых 5-ти значений
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
# краткий обзор об имеющейся базе данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Общее количество клиентов составляет 21 525 человек. По каждому клиенту имеются 12 показателей.

In [4]:
# подсчет количества пропущенных значений, сортировка по убыванию
data.isna().sum().sort_values(ascending=False)

total_income        2174
days_employed       2174
purpose                0
debt                   0
income_type            0
gender                 0
family_status_id       0
family_status          0
education_id           0
education              0
dob_years              0
children               0
dtype: int64

Тем самым, пропущенные значения имеют место в двух переменных: (а) трудовой стаж, (б) ежемесячный доход. Количество пропущенных значений в абсолютном выражении составляет 2 174 наблюдения.

In [5]:
# вычисление доли пропущенных значений в общей базе данных
na_ratio = data.isna().sum().sort_values(ascending=False)[0] / data.shape[0]
# вывод результатов
print('Доля пропущенных значений составляет {:.2%}'.format(na_ratio))

Доля пропущенных значений составляет 10.10%


In [6]:
# посмотрим клиентов у которых отсутствует информация по количеству трудового стажа в днях
data.sort_values(by = 'days_employed', ascending=False).tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


Клиенты по которым отсутствует информация по трудовому стажу и ежемесячному доходу довольно сильно различаются между собой не только по типу занятости, но и по возрасту.

In [7]:
# опираясь на полученное количество пропущенных значений (2 174 наблюдения),
# выделим отдельную базу данных по которым отсутствует информация по трудовому стажу и ежемесячном доходе
df_na = data.sort_values(by='days_employed', ascending=False).tail(2174)
# далее, сгруппируем данных клиентов по типу дохода
df_na.groupby('income_type')['income_type'].count().sort_values(ascending=False)

income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Клиенты, тип дохода которых "сотрудник" составляют значительную часть наблюдений по которым отсутствует информация. Отсутствие у пенсионеров трудового стажа может быть технической причиной: у нынешнего поколения пенсионеров просто может быть не сохранилась трудовая книжка. В отношении 'сотрудников' и 'компаньонов' дело может обстоять иначе: у сотрудников могут быть низкие доходы, у партнера может и не быть вообще официального ежемесячного дохода. На то и он партнер, чтобы выполнять лишь вспомогательную функцию.

### Вывод

- Общее количество клиентов составляет 21 525 человек. По каждому клиенту имеются 12 показателей.
- Доля пропущенных значений в общей базе данных составляет 10 процентов.
- Градация клиентов по которым отсутствуют данные по трудовому стажу в днях довольно обширная и следовательно заменять на средние величины или медиану кажется не самым подходящим вариантом.
- Причина наличия пропущенных значений может быть как случайной, так и неслучайной.
- С одной стороны, клиенты с низким ежемесячным доходом вероятнее всего будут характеризоваться как неплатежеспособные заемщики, что является препятствием к получению кредита. Осознавая данное обстоятельство клиенты могут не декларировать стаж и доход дабы не относить себя к числу неплатежеспособных заемщиков.
- С другой стороны, пропущенные наблюдения могут быть обусловлены техническими причинами заполнения значений.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [8]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Ранее, также было указано, что пропущенные наблюдения присутствуют в двух показателях: трудовой стаж, ежемесячный доход. Взглянем на данные переменные более подробно.

Изучим переменную "трудовой стаж".

In [9]:
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

В отношении трудового стажа можно наблюдать следующее: (а) минимальный стаж представляет собой отрицательную величину, (б) максимальный стаж составляет более 1100 лет, (в) средний стаж составляет 172 года, (г) весьма большой разброс вокруг среднего значения: плюс минус 140 тыс. дней, (д) медиана также является отрицательной величиной.\
Промежуточный вывод: действительно на место пропущенных значений не подходит среднее значение, медиана.\
Весьма забавная выборочная совокупность!\
Подобные феномены могут быть обусловлены ошибками при заполнении. Для предоления данных проблем выполним следующее: (1) конвертируем отрицательные значения в положительные, (2) поделим величины стажа более 25 000 дней (около 68-ми лет) на 100. \

In [10]:
# конвертируем отрицательный трудовой доход в положительный
data['days_employed'] = abs(data['days_employed'])

In [11]:
# определим следующий цикл (по индексам):
# для каждой строчки исходной выборочной совокупности
for i in range(len(data)):
    try:
        # если трудовой стаж более 20 000 дней, то
        if data.loc[i,'days_employed'] > 25000:
            # делим данный стаж на 100
            data.loc[i, 'days_employed'] = data.loc[i, 'days_employed'] / 100
    except:
        print('Ошибка! Возможно стоит пересмотреть функцию.')

In [12]:
data['days_employed'].describe()

count    19351.000000
mean      2583.921756
std       2149.032624
min         24.141633
25%        927.009265
50%       2194.220567
75%       3658.750303
max      18388.949901
Name: days_employed, dtype: float64

In [13]:
# переведем основные числовые характеристики в года
print('Наибольший трудовой стаж составляет {:.2f} лет'.format(data['days_employed'].max() / 365))
print('Наименьший трудовой стаж составляет {:.2f} лет'.format(data['days_employed'].min() / 365 ))
print('Медиана трудового стажа составляет {:.2f} лет'.format(data['days_employed'].median() / 365 ))
print('Разброс вокруг среднего стажа работы составляет {:.2f} лет'.format(data['days_employed'].std() / 365 ))

Наибольший трудовой стаж составляет 50.38 лет
Наименьший трудовой стаж составляет 0.07 лет
Медиана трудового стажа составляет 6.01 лет
Разброс вокруг среднего стажа работы составляет 5.89 лет


In [14]:
# определим клиентов, у которых отсутствует информация по трудовому стажу
df_na['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Далее, из исходной выборочной совокупности выделим пять подвыборок: сотрудники, компаньоны, госслужащие, предприниматели, пенсионеры.

In [15]:
employee_mean_days_employed = data.loc[data['income_type']=='сотрудник', 'days_employed'].mean()
companion_mean_days_employed = data.loc[data['income_type']=='компаньон', 'days_employed'].mean()
pensioner_mean_days_employed = data.loc[data['income_type']=='пенсионер', 'days_employed'].mean()
state_employee_mean_days_employed = data.loc[data['income_type']=='госслужащий', 'days_employed'].mean()
entrepreneur_mean_days_employed = data.loc[data['income_type']=='предприниматель', 'days_employed'].mean()

Действительно, замена пропущенных значений на общее среднее является неподходящим вариантом: трудовой стаж пенсионера и молодого сотрудника могут сильно различаться; также трудовой стаж между опытным гос.служащим и молодым предпринимателем также может характеризоваться большим разрывом.\
С учетом данной гетерогенности, стоит заменить на средние величины по каждой группе клиентов: например, можно предположить что ежемесячный доход между пенсионерами может и сильно не отличаться.\
Таким образом, пропущенные значения заменяются не на **общее** среднее, а на **средние величины по каждому типу клиентов согласно виду занятости**. 

In [16]:
df_na.loc[df_na['income_type']=='сотрудник', 'days_employed'] = employee_mean_days_employed
df_na.loc[df_na['income_type']=='компаньон', 'days_employed'] = companion_mean_days_employed
df_na.loc[df_na['income_type']=='пенсионер', 'days_employed'] = pensioner_mean_days_employed
df_na.loc[df_na['income_type']=='госслужащий', 'days_employed'] = state_employee_mean_days_employed
df_na.loc[df_na['income_type']=='предприниматель', 'days_employed'] = entrepreneur_mean_days_employed

Далее, перейдем к переменной "ежемесячные доходы".

In [17]:
data['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

Касательно ежемесячных расходов, то наблюдаем следующее: (а) минимальный ежемесячный доход составляет 20 667 у.е., (б) максимальный ежемесячный доход равен 2 265 604 у.е., (в) средний ежемесячный доход составляет 167 422 у.е., (г) разброс вокруг среднего приблизительно 102 971 у.е.\
Промежуточный вывод: на место пропущенных значений не подходит **общее** среднее значение и медиана.\
Аналогично, вместо **общего** среднего, заменяем на **частные** средние величины.

In [18]:
employee_mean_income = data.loc[data['income_type']=='сотрудник','total_income'].mean()
companion_mean_income = data.loc[data['income_type']=='компаньон','total_income'].mean()
pensioner_mean_income = data.loc[data['income_type']=='пенсионер','total_income'].mean()
state_employee_mean_income = data.loc[data['income_type']=='госслужащий','total_income'].mean()
entrepreneur_mean_income = data.loc[data['income_type']=='предприниматель','total_income'].mean()

In [19]:
df_na.loc[df_na['income_type']=='сотрудник', 'total_income'] = employee_mean_income
df_na.loc[df_na['income_type']=='компаньон', 'total_income'] = companion_mean_income
df_na.loc[df_na['income_type']=='пенсионер', 'total_income'] = pensioner_mean_income
df_na.loc[df_na['income_type']=='госслужащий', 'total_income'] = state_employee_mean_income
df_na.loc[df_na['income_type']=='предприниматель', 'total_income'] = entrepreneur_mean_income

In [20]:
df_na.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Далее полученные величины заменим в исходную выборочную совокупность.

In [21]:
# находим размерность выборки без пропущенных переменных
df_without_na_shape = data.shape[0] - df_na.shape[0]
df_without_na_shape

19351

In [22]:
# поскольку клиенты по которым отсутствует информация по трудовому стажу одновременно и отсутствует информация
# по ежемесячному доходу, то отсортировав по одному признаку (в частности, по трудовому доходу), также и 
# отсортируем по второму признаку (в частности, по ежемесячному доходу)
df_without_na = data.sort_values(by='days_employed')[:df_without_na_shape]

In [23]:
# далее, соединим первую выборку без пропущенных значений с выборкой, в которой ранее заменили пропущенные 
# значения на средние величины по каждому типу клиентов согласно их виду занятости (соединяем по строчке)
df = pd.concat([df_without_na, df_na],axis=0)

In [24]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Получили более приятную картину. Однако, остается другая проблема - дублированные величины - на разрешение которой и посвящен следующий раздел (точнее, через один раздел).

### Вывод

Таким образом, на данном этапе стоит отметить следующее: (а) общее количество пропущенных переменных составляет 2174 наблюдения, что в относительном выражении равно 10-ти процентам, (б) преимущественная часть пропущенных значений наблюдается в двух показателях таких как "трудовой доход в днях", "ежемесячный доход", причем пропущенные значения в одном показателе, также одновременно присутствуют (пропущенные переменные) в другом показателе.\
С учетом разнородности величин как по трудовому стажу, так и по ежемесячному доходу, находились не **общие** средние, а напротив, по каждому типу клиента, согласно трудовому стажу и ежемесячному доходу. Далее, полученные величины являлись заменой для пропущенных переменных, так как можно предполагать то, что клиент внутри своей когорты (например пенсионер с другим пенсионером) сильно не отличается как по трудовому стажу, так и по ежемесячному доходу.

### Замена типа данных

In [25]:
# вновь проведем краткий обзор по выборочной совокупности
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 17437 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


К вещественными типам относятся следующие переменные: трудовой стаж в днях, ежемесячный доход.

In [26]:
# используем метод astype для преобразования вещественных типов данных в целочисленные
# как для трудового стажа в днях,
df['days_employed'] = df['days_employed'].astype(int)
# так и для ежемесячного дохода
df['total_income'] = df['total_income'].astype(int)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 17437 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

На данный момент имеется база данных в которой количественные показатели имеют целочисленный формат.

### Обработка дубликатов

In [28]:
# проверка наличия дублированных наблюдений в выборочной совокупности
df.duplicated().sum()

54

Наблюдаются 54 дублированных наблюдения.

Так как некоторые заполненные переменные могут отличаться от друг друга лишь регистром, приведем столбцы "образование", "семейный статус", "тип занятости", "цель получения кредита" к нижнему регистру и вновь проверим наличие дублированных наблюдений.

In [29]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()
df['gender'] = df['gender'].str.upper()

In [30]:
df.duplicated().sum()

71

In [31]:
# аккуратно удалим клонов из выборочной совокупности, также переобозначим индексы с удалением лишних индексов
df = df.drop_duplicates().reset_index(drop=True)

In [32]:
# вновь проверим наличие дублированных наблюдений
df.duplicated().sum()

0

### Вывод

На данном этапе стоит отметить следующее:
- на первой проверке выявили 54-х дублированных наблюдения. Однако с учетом приведения столбцов "образование", "семейный статус", "тип занятости", "цель получения кредита", выявились дополнительные дублированные наблюдения, что в совокупности составило 71-но наблюдение. 
- Далее, убрали дублированные наблюдения из выборочной совокупности.
- Наличие дублированных наблюдений может быть обусловлено ...

### Лемматизация

In [33]:
# загрузка необходимой библиотеки
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer

In [34]:
# инициализация
m = Mystem()
russian_stemmer = SnowballStemmer('russian')

In [35]:
# определяем пустой лист, в который будут размещены лемматизированные слова
lemmas = []

# каждый элемент в тексте
for item in df['purpose']:
    try:
        # приведем к лемматизированному виду и добавим к пустому листу
        lemma = m.lemmatize(item)
        lemmas.append(lemma)
    except:
        print('Возникла ошибка! Стоит пересмотреть цикл')

In [36]:
# вывод первых 5-ти элементов списка с лемматизированными словами
lemmas[:5]

[['высокий', ' ', 'образование', '\n'],
 ['получение', ' ', 'дополнительный', ' ', 'образование', '\n'],
 ['свадьба', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['получение', ' ', 'образование', '\n']]

In [37]:
# уберем лишние пробелы в каждой строчке лемматизированного списка
lemmas = [''.join(item) for item in lemmas]

In [38]:
print('Уникальные значения в перечне лемматизированных причин:')
# посмотрим на уникальные значения в спике лемматизированных слов
for item in pd.Series(lemmas).sort_values().unique():
    print('* ',end='')
    print(item, end ='')

Уникальные значения в перечне лемматизированных причин:
* автомобиль
* высокий образование
* дополнительный образование
* жилье
* заниматься высокий образование
* заниматься образование
* на покупка автомобиль
* на покупка подержать автомобиль
* на покупка свой автомобиль
* на проведение свадьба
* недвижимость
* образование
* операция с жилье
* операция с коммерческий недвижимость
* операция с недвижимость
* операция со свой недвижимость
* покупка жилой недвижимость
* покупка жилье
* покупка жилье для сдача
* покупка жилье для семья
* покупка коммерческий недвижимость
* покупка недвижимость
* покупка свой жилье
* получение высокий образование
* получение дополнительный образование
* получение образование
* приобретение автомобиль
* профильный образование
* ремонт жилье
* свадьба
* свой автомобиль
* сделка с автомобиль
* сделка с подержанный автомобиль
* строительство жилой недвижимость
* строительство недвижимость
* строительство собственный недвижимость
* сыграть свадьба


Каких только не бывает причин получения кредита: начиная от самого необходимого, например приобретение жилья, до более роскошного, например проведение свадьбы, что тоже хорошо :)\
Таким образом, основными целями получениям кредита являются (а) приобретение автомобиля, (б) получение образования, (в) проведение свадьбы, (г) покупка жилья и недвижимость (для собственного проживания), (д) коммерческая недвижимость (для перепродажи или сдачи в аренду).

In [39]:
def lemma_for_purpose(lemmas):
    list_of_lemmas = []
    
    for i in range(len(lemmas)):
        # сначала определим более явные цели
        if 'образование' in lemmas[i]:
            list_of_lemmas.append('образование')
    
        elif 'свадьба' in lemmas[i]:
            list_of_lemmas.append('свадьба')
            
        elif 'автомобиль' in lemmas[i]:
            list_of_lemmas.append('автомобиль')
        # 
        elif ('жилье' in lemmas[i]) or ('собственный' in lemmas[i]) or ('жилой' in lemmas[i]) or ('недвижимость' in lemmas[i]):
            list_of_lemmas.append('недвижимость')
        
        # проверка на оставшиеся цели помимо вышеуказанных
        else:
            list_of_lemmas.append('другое')
            
    return list_of_lemmas

In [40]:
try:
    # присоединим леммы в выборочную совокупность
    df['lemma'] = lemma_for_purpose(lemmas)
    # вывод первых пяти наблюдений
    df.head()
except:
    print('Ошибка! Стоит пересмотеть функцию "lemma_for_purpose"')

In [41]:
# подсчет уникальных лемм из перечня целей получения кредита
df['lemma'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: lemma, dtype: int64

### Выводы

Завершая данный раздел, стоит отметить следующее:
- постарались выделить леммы для целей получения кредита в выборочной совокупности,
- выявились основные 4 категории к числу которых относятся: (1) недвижимость ( как для собственного проживания, так и для осуществления коммерческих операций), (2) автомобиль, (3) образование, (4) свадьба.
- Действительно, в результате подсчета уникальных значений, не было выявлено категории "другое", т.е. выделенные 4 категории охватывают весь перечень целей получения кредита.
- Лидером целей получения кредита является приобретение недвижимости.
- Меньше всего клиенты обращаются за приобретением кредита для проведения свадьбы.

### Категоризация данных

In [42]:
# определим основные числовые характеристики ежемесячных доходов клиентов
df['total_income'].describe()

count    2.145400e+04
mean     1.674316e+05
std      9.806060e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.518870e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

Касательно ежемесячных расходов, наблюдаем следующее:\
(а) минимальный ежемесячный доход составляет 20 667 у.е.,\
(б) средний ежемесячный доход составляет 167 422 у.е.,\
(в) максимальный ежемесячный доход равен 2 265 604 у.е.,\
(г) разброс вокруг среднего приблизительно 102 971 у.е.\
Опираясь на данную информацию, можно разделить клиентов на три категории в зависимости от величины ежемесячных доходов:\
(1) "низкий доход": от наименьшей величины до среднего значения (не включено). Медиана скорее не подойдет, поскольку в распределении наблюдается левый толстый хвост, что может привести к смещению;\
(2) "средний доход": от среднего (включительно) до величины равной 75-ти процентам (не включительно) распределения доходов;\
(3) "высокий доход": от 75-ти процентов распределения доходов до максимального значения.

In [43]:
# вывод величины на 75-ти процентном распределения
third_quartile = df['total_income'].describe()[6]
# средний ежемесячный доход
mean_income = df['total_income'].mean()

In [44]:
def income_category(income):
    '''
    income: столбец со значениями доходов
    output: классификация клиентов на три класса в зависимости от
    величины ежемесячных доходов
    '''     
    if income < mean_income:
        return 'низкий доход'
    elif mean_income <= income < third_quartile:
        return 'средний доход'
    elif income >= third_quartile:
        return 'высокий доход'

In [45]:
# воспользуемся функцией классификации клиентов в зависимости от величины ежемесячного дохода
try:
    df['income_category'] = df['total_income'].apply(income_category)
except:
    print('Ошибка! Стоит пересмотреть фунцию классификации в зависимости от ежемесячного дохода')

In [46]:
# произведем подсчет уникальных значений каждой категории клиентов
# в зависимости от величины ежемесячного дохода
df['income_category'].value_counts()

низкий доход     13350
высокий доход     5410
средний доход     2694
Name: income_category, dtype: int64

Далее, осуществим классификацию клиентов в зависимости от возраста.

In [47]:
# определим числовые характеристики возроста клиентов
df['dob_years'].describe()

count    21454.000000
mean        43.271231
std         12.570822
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

В отношении возраста наблюдаем следующее:\
(а) средний возраст клиента составляет 43 года;\
(б) разброс вокруг среднего возраста составляет 12 лет;\
(в) самому старшему клиенту 75 лет.

In [48]:
# определим функцию классификации клиентов в зависимости от возраста
def age_category(age):
    '''
    age: возраст клиентов
    output: классификация клиентов на три категории в зависимости от возраста:
    (1) младше 19 лет: дети
    (2) от 19 до 64 лет: взрослые
    (3) старше 64 лет: пенсионеры
    '''
    if age < 19:
        return 'дети'
    elif 19 <= age < 64:
        return 'взрослые'
    else:
        return 'пенсионеры'

In [49]:
# используем функцию классификации клиентов в зависимости от возраста
try:
    df['age_category'] = df['dob_years'].apply(age_category)
except:
    print('Ошибка! Стоит пересмотреть функцию классификации клиентов в зависимости от возраста')

In [50]:
# произведем подсчет возрастных категорий
df['age_category'].value_counts()

взрослые      20198
пенсионеры     1155
дети            101
Name: age_category, dtype: int64

Возраст значительной части клиентов являются взрослыми людьми.

### Вывод

Завершая раздел "категоризация данных", стоит отметить следующее:\
(а) произвели классификацию клиентов в зависимости от ежемесячного дохода на три категории: с низким, средним и высоким доходами, соответственно. Значительная часть клиентов имеет доход ниже средней величины;\
(б) произвели классификацию клиентов в зависимости от возраста на три категории: дети - младше 19 лет, взрослые - от 19 до 64 лет, пенсионеры - старше 64 лет. Значительная часть клиентов попадают в категорию взрослых.\
Также есть и те, которые несмотря на молодой возраст обращаются к кредитным учреждениям. Помимо слишком молодых, пенсионеры также участвуют в долговых отношениях.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [51]:
# определим числовые характеристики количества детей клиентов
df['children'].describe()

count    21454.000000
mean         0.539946
std          1.383444
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

Видим наличие отрицательного количества детей. Переведем отрицательное количество детей в положительное. Пускай будет больше детей...но, есть и крайние случаи: 20 детей!

In [52]:
# произведем подсчет клиентов в зависимости от количества детей
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Вдобавок, видим резкий переход от 5-ти детей к крайне большому количеству в размере 20 детей!

In [53]:
# посмотрим у кого же получилось отрицательное число детей
df.loc[df['children']==-1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,income_category,age_category
705,-1,195,31,среднее,1,женат / замужем,0,F,сотрудник,0,145577,ремонт жилью,недвижимость,низкий доход,взрослые
1252,-1,268,23,среднее,1,в разводе,3,F,компаньон,0,92257,покупка жилой недвижимости,недвижимость,низкий доход,взрослые
1860,-1,370,27,среднее,1,женат / замужем,0,F,компаньон,0,164591,покупка жилья для сдачи,недвижимость,низкий доход,взрослые
1865,-1,372,43,среднее,1,женат / замужем,0,M,сотрудник,0,155588,сделка с подержанным автомобилем,автомобиль,низкий доход,взрослые
2385,-1,457,33,среднее,1,женат / замужем,0,F,сотрудник,1,149641,автомобиль,автомобиль,низкий доход,взрослые
2755,-1,526,51,среднее,1,женат / замужем,0,F,сотрудник,0,146928,операции с коммерческой недвижимостью,недвижимость,низкий доход,взрослые
3230,-1,617,38,среднее,1,не женат / не замужем,4,M,сотрудник,0,122205,строительство жилой недвижимости,недвижимость,низкий доход,взрослые
3463,-1,661,32,высшее,0,женат / замужем,0,F,сотрудник,0,137405,покупка жилой недвижимости,недвижимость,низкий доход,взрослые
4666,-1,895,37,среднее,1,женат / замужем,0,F,компаньон,0,214814,жилье,недвижимость,высокий доход,взрослые
4700,-1,901,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375,операции со своей недвижимостью,недвижимость,высокий доход,взрослые


In [54]:
# конвертируем отрицательное количество детей в положительное
df['children'] = abs(df['children'])

In [55]:
# посмотрим на цели получения кредита многодетных клиентов
df.loc[df['children']==20]['purpose'].value_counts()

строительство собственной недвижимости    6
на покупку подержанного автомобиля        6
высшее образование                        5
сыграть свадьбу                           4
на проведение свадьбы                     4
сделка с подержанным автомобилем          4
операции с жильем                         4
операции с коммерческой недвижимостью     4
жилье                                     4
покупка коммерческой недвижимости         3
покупка жилья                             3
свой автомобиль                           3
дополнительное образование                3
покупка недвижимости                      2
образование                               2
профильное образование                    2
покупка жилой недвижимости                2
ремонт жилью                              2
недвижимость                              1
автомобили                                1
свадьба                                   1
автомобиль                                1
покупка жилья для семьи         

Если бы на самом деле было 20 детей, то скорее данные клиенты обращались бы за кредитов с целью приобретения жилья для благоприятного проживания большой семьи. Однако, исходя из вышеприведенной таблицы, можно скорее предположить то, что наличие многодетных детей обусловлено технической ошибкой при заполнении. Также, как было указано ранее, наблюдаем резкий переход от 5 к 20 детям!

In [56]:
# вероятнее всего в контексте детей лишним в числе 20 является 0
df.loc[df['children']==20] = df.loc[df['children']==2]

In [57]:
# вновь, произведем подсчет количества детей по клиентам
df['children'].value_counts()

0.0    14091
1.0     4855
2.0     2052
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

In [58]:
# произведем подсчет наличия/отсутствия задолженностей в зависимости от количества детей
df.groupby('children')['debt'].value_counts().unstack()

debt,0.0,1.0
children,,
0.0,13028.0,1063.0
1.0,4410.0,445.0
2.0,1858.0,194.0
3.0,303.0,27.0
4.0,37.0,4.0
5.0,9.0,NaN


С одной стороны, значительная часть клиентов не имеют детей. С другой стороны, многодетные клиенты несмотря на большое семейство редко обращаются за приобретением кредита...даже если и приобретают, то зачастую своевременно выплачивают долговые обязательства.

In [59]:
# определим сводную таблицу количества клиентов с задолженностью в разрезе количества детей
debt_children = df.pivot_table(index='children', values='debt')
debt_children

,debt
children,
0.0,0.075438
1.0,0.091658
2.0,0.094542
3.0,0.081818
4.0,0.097561
5.0,0.000000


In [60]:
# переведем полученные значения наличия задолженности клиентов в разрезе количества детей в проценты
debt_children['debt_presence_ratio'] = debt_children['debt'] * 100
debt_children

,debt,debt_presence_ratio
children,,
0.0,0.075438,7.543822
1.0,0.091658,9.165808
2.0,0.094542,9.454191
3.0,0.081818,8.181818
4.0,0.097561,9.756098
5.0,0.000000,0.000000


### Вывод

Согласно вышеприведенной таблице, строгой зависимости между количеством детей и возвратом кредита в срок не наблюдается: клиенты как без детей, так и многодетные клиенты более чем в 90% случаев своевременно выполняют долговые обязательства.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [61]:
# осуществим общий подсчет наличия/отсутствия задолженностей у клиентов в разрезе семейного положения
df.groupby('family_status')['debt'].value_counts().unstack()

debt,0.0,1.0
family_status,,
в разводе,1109,84
вдовец / вдова,892,63
гражданский брак,3754,385
женат / замужем,11362,928
не женат / не замужем,2528,273


С одной стороны, наибольшая часть клиентов являются женатыми / замужними. С другой стороны, клиенты, потерявшие супруга / супругу составляют наименьшую долю в выборочной совокупности.

In [62]:
# осуществим подсчет количества клиентов с задолженностью в разрезе семейного положения
family_status_debt = df.pivot_table(index='family_status', values='debt')
family_status_debt

,debt
family_status,
в разводе,0.070411
вдовец / вдова,0.065969
гражданский брак,0.093018
женат / замужем,0.075509
не женат / не замужем,0.097465


In [63]:
# переведем полученные значения наличия задолженности в разрезе семейного положения в проценты
family_status_debt['debt_presence_ration'] = family_status_debt['debt'] * 100
family_status_debt

,debt,debt_presence_ration
family_status,,
в разводе,0.070411,7.041073
вдовец / вдова,0.065969,6.596859
гражданский брак,0.093018,9.301764
женат / замужем,0.075509,7.550854
не женат / не замужем,0.097465,9.746519


Строгой зависимости между семейный положением и возвратом кредита в срок не наблюдается. Поскольку как не женатые и не замужние, так и женатые и замужние клиенты в большинстве случаев не имеют задолженности. Аналогично, клиенты, которые в разводе, или потерявшие супруга или супругу также своевременно выплачивают долговые обязательства. Клиенты, состоящие в гражданском браке также зачастую не имеют задолженности.\
Таким образом, значительная часть клиентов независимо от семейного положения в более 90% случаев не имеет проблем со своевременным выполнением долговых обязательств.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [64]:
# осуществим подсчет наличия/отсутствия задолженности в зависимости от типа доходов:
df.groupby('income_category')['debt'].value_counts().unstack()

debt,0.0,1.0
income_category,,
высокий доход,5015,373
низкий доход,12172,1132
средний доход,2458,228


Наибольшая доля клиентов имеет доходы ниже среднего уровня. Наверное, поэтому и обращаются за дополнительными заемными средства.

In [65]:
# определим сводную таблицу количества клиентов с задолженностью в разрезе уровня доходов
income_category_debt = df.pivot_table(index='income_category', values='debt')
income_category_debt

,debt
income_category,
высокий доход,0.069228
низкий доход,0.085087
средний доход,0.084885


In [66]:
# переведем полученные значения наличия задолженностей в разрезе уровня дохода в проценты
income_category_debt['debt_presence_ratio'] = income_category_debt['debt'] * 100
income_category_debt

,debt,debt_presence_ratio
income_category,,
высокий доход,0.069228,6.922791
низкий доход,0.085087,8.508719
средний доход,0.084885,8.488459


### Вывод

Строгой зависимости между уровнем дохода и возвратом кредита в срок нет: клиенты как с низкими, так и со средними и высокими доходами в более 90% случаев своевременно выполняют долговые обязательства.

- Как разные цели кредита влияют на его возврат в срок?

In [67]:
# осуществим общий подсчет наличия/отсутсвия задолженности в разрезе целей получения кредита
df.groupby('lemma')['debt'].value_counts().unstack()

debt,0.0,1.0
lemma,,
автомобиль,3889,401
недвижимость,9995,780
образование,3629,369
свадьба,2132,183


Клиенты целью получения кредита которых является приобретение недвижимости составляют наибольшую долю.

In [68]:
# определим сводную таблицу задолженностей клиентов в разрезе целей получения кредита
purpose_lemma_debt = df.pivot_table(index='lemma', values='debt')
purpose_lemma_debt

,debt
lemma,
автомобиль,0.093473
недвижимость,0.072390
образование,0.092296
свадьба,0.079050


In [69]:
# полученные величины наличия задолженностей в разрезе целей получения кредита переведем в проценты
purpose_lemma_debt['debt_presence_ratio'] = purpose_lemma_debt['debt'] * 100
purpose_lemma_debt

,debt,debt_presence_ratio
lemma,,
автомобиль,0.093473,9.347319
недвижимость,0.072390,7.238979
образование,0.092296,9.229615
свадьба,0.079050,7.904968


### Вывод

Строгой зависимости между целями получения кредита и наличием/отсутствием задолженностей не наблюдается: начиная от клиентов целью которых является приобретение необходимого актива как жилье, так и для дополнительных торжественных мероприятий, в частности, проведение свадьбы в более 90% случаев своевременно выполняют долговые обязательства.

### Шаг 4. Общий вывод

### Таким образом, завершая данный проект, стоит отметить следующие моменты:

* общая выборочная совокупность составляет 21 525 клиентов, по каждому из которых имеются 12 показателей, начиная от трудового стажа и заканчивая количеством детей.
* пропущенные значения, преимущественно, наблюдаются в двух показателях: трудовой стаж в днях, ежемесячный доход. Причины данных пропусков могут нести как технический характер (ошибки при заполнении), так и неслучайный характер (клиенты с низкими доходами могут осознавать что малый достаток препятствует получению кредита в силу чего не декларируют фактический доход). В отношении пропуска по трудовому стажу, могут быть следующие причины: у пенсионеров могут не сохраниться трудовые книжки, что затрудняет оценивание фактического стажа; 'партнеры' могут быть заняты на неофициальных сферах деятельности: семейный бизнес, товарищество (в которых они участвуют как партнеры).
* доля пропущенных значений в общей выборочной совокупности составляет 10.10%;
* в выборочной совокупности присутсвует 54 дублированных значения, однако при приведении столбцов "образование","семейный статус", "тип занятости", "цель получения кредита", общее количество дублированных значений (клонов) составило 71 наблюдение;
* выделены следующие **леммы** причин получения кредитов: (а) недвижимость, (б) образование, (в) свадьба, (г) автомобиль.

#### Ответы на вопросы:
* строгой  зависимости между количеством детей и возвратом кредита не наблюдается: клиенты как без детей, так и многодетные (5 детей) в 90% случаев своевременно выполняют долговые обязательства;
* аналогично, строгой зависимость между семейный положением и возвратом кредита в срок не наблюдается: клиенты, которые как и состоящие, так и не состоящие в официальном браке в 90% случаев не имеют задолженности. Аналогичная ситуация имеет место в отношении клиентов, состоящих в гражданском браке.
* строгой зависимости между уровнем доходам и выплатой кредита в срок не наблюдается: клиенты как с низкими, так и со средними и высокими ежемесячными доходами в 90% случаев своевременно выплачивают кредит;
* клиенты как приобретающие кредит для покупки необходимого актива как недвижимость, так и для проведения торжественного мероприятия как свадьба в более 90% случаев своевременного выполняют долговые обязательства.